In [1]:
%%capture
! pip install pyautogen 
! pip install tempfile


In [21]:
from autogen.retrieve_utils import TEXT_FORMATS
import autogen
from autogen.coding import DockerCommandLineCodeExecutor
from autogen.agentchat.contrib.agent_builder import AgentBuilder


In [22]:
ollama_api_endpoint = "http://localhost:11434" # Development endpoint port forwarded
# ollama_api_endpoint = "http://ollama.ollama.svc.cluster.local:11434" #Production endpoint for in cluster use

# Create an autogen config list
config_list = [
  {
    "model": "gemma:latest",
    "base_url": ollama_api_endpoint + "/v1",
    "api_key": "ollama",
  }
]


In [23]:
# create a docker executor to execute code
executor = DockerCommandLineCodeExecutor(
    image="python:3.12-slim",  # Execute code using the given docker image name.
    timeout=30,  # Timeout for each code execution in seconds.
    work_dir="coding/",  # Use the temporary directory to store the code files.
)


In [24]:
# create an AssistantAgent named "assistant"
assistant = AssistantAgent(
    name="assistant",
    llm_config={
        "cache_seed": 41,  # seed for caching and reproducibility
        "config_list": config_list,  # a list of OpenAI API configurations
        "temperature": 0,  # temperature for sampling
    },  # configuration for autogen's enhanced inference API which is compatible with OpenAI API
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={"executor": executor},  # Use the docker command line code executor.
)
# the assistant receives a message from the user_proxy, which contains the task description
chat_res = user_proxy.initiate_chat(
    assistant,
    message="""
    Write me a simple Flask application which serves a static HTML app.
    The app should have a single route that serves the HTML file.
    The HTML file should have a button that when clicked, sends a POST request to the server (to http://ollama.ollama.svc.cluster.local:11434). 
    The server is an Ollama endpoint which uses an OpenAI API compatible model to generate a response.
    """,
    summary_method="reflection_with_llm",
)

user_proxy (to assistant):


    Write me a simple Flask application which serves a static HTML app.
    The app should have a single route that serves the HTML file.
    The HTML file should have a button that when clicked, sends a POST request to the server (to http://ollama.ollama.svc.cluster.local:11434). 
    The server is an Ollama endpoint which uses an OpenAI API compatible model to generate a response.
    

--------------------------------------------------------------------------------
assistant (to user_proxy):

**Python Code:**

```python
# filename: simple_flask_app.py

from flask import Flask, render_template, request

app = Flask(__name__)

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/submit", methods=["POST"])
def submit():
    # Get the user's input
    data = request.form

    # Send the input to the Ollama endpoint
    response = requests.post("http://ollama.ollama.svc.cluster.local:11434", data=data)

    # Return the Ollama en